<a href="https://colab.research.google.com/github/kalai2315/Report_Generator_Agent/blob/main/Report_Generator_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini project 2: Building a Report Generator Agent

**Branching and Merging Chains with LCEL**

Branching and Merging Chains are concepts used in LangChain and LCEL (LangChain Expression Language) to handle conditional logic and parallel or selective execution of chains based on input.

In [1]:

%pip install -U --quiet langchain langchain-google-genai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:

import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [3]:

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest")

In [4]:

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser

In [5]:
description_prompt = ChatPromptTemplate.from_template(
    """Generate a two line description for the given topic:
    {topic}
    """
)
description_chain = description_prompt | llm | StrOutputParser()

In [6]:
pros_prompt = ChatPromptTemplate.from_template(
    """Generate three bullet points talking about the pros for the given topic:
    {topic}
    """
)

pros_chain = pros_prompt | llm | StrOutputParser()


In [7]:
cons_prompt = ChatPromptTemplate.from_template(
    """Generate three bullet points talking about the cons for the given topic:
    {topic}
    """
)

cons_chain = cons_prompt | llm | StrOutputParser()


In [10]:
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

branch_chain = (RunnableParallel(
    topic=itemgetter("topic"),
    description=(description_chain),
    pros=pros_chain,
    cons=cons_chain,
))

In [11]:
branch_chain.invoke({"topic": "Artificial Intelligence"})

{'topic': 'Artificial Intelligence',
 'description': "Artificial intelligence (AI) simulates human intelligence in machines, enabling them to learn, reason, and solve problems.  It's rapidly transforming industries and impacting nearly every aspect of modern life.",
 'pros': '* **Increased Efficiency and Productivity:** AI automates tasks, analyzes data at speeds far exceeding human capabilities, and optimizes processes across various industries, leading to significant gains in efficiency and productivity.\n\n* **Improved Decision-Making:** AI algorithms can analyze vast datasets to identify patterns and trends invisible to humans, providing valuable insights for better and more informed decision-making in areas like healthcare, finance, and business strategy.\n\n* **Solving Complex Problems:** AI can tackle complex problems that are beyond human capacity, such as predicting natural disasters, developing new medicines, and optimizing resource allocation in challenging environments.',
 

In [17]:
merge_prompt = ChatPromptTemplate.from_template(
    """create a report about {topic} with the following information:
    Description: {description}
    Pros: {pros}
    Cons: {cons}

    Report should be in the following format:
    Topic: <name of the topic>
    Description: <description of the topic>
    pros and Cons:
    <Table with two columns showing the 3 pros and cons of the topic>
    """
)

merge_chain = merge_prompt | llm | StrOutputParser()


In [18]:
final_chain = (branch_chain | merge_chain)


In [19]:
response = final_chain.invoke({"topic": "Artificial Intelligence"})

In [20]:
from IPython.display import display, Markdown

display(Markdown(response))

Topic: Artificial Intelligence (AI)

Description: Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. It encompasses learning, reasoning, problem-solving, and perception, enabling computers to perform tasks that typically require human intelligence.


Pros and Cons:

| Pros                                      | Cons                                           |
|-------------------------------------------|-------------------------------------------------|
| Increased Efficiency and Productivity     | Job displacement                                 |
| Improved Decision-Making                   | Bias and discrimination                           |
| Solving Complex Problems                   | Lack of transparency and accountability          |